## Ch 16: Structs and Functions

Now, let's practice writing functions that take programmer-defined objects as parameters and return them as results.

In [2]:
mutable struct MyTime
    
    hour
    minute
    second
    
end

### Exercise 16-1

Write a function called printtime that takes a MyTime object and prints it in the form hour:minute:second . The
@printf macro of the StdLib module Printf prints an integer with the format sequence "%02d" using at least two
digits, including a leading zero if necessary.

In [3]:
using Printf

In [4]:
?@printf

```
@printf([io::IOStream], "%Fmt", args...)
```

Print `args` using C `printf` style format specification string, with some caveats: `Inf` and `NaN` are printed consistently as `Inf` and `NaN` for flags `%a`, `%A`, `%e`, `%E`, `%f`, `%F`, `%g`, and `%G`. Furthermore, if a floating point number is equally close to the numeric values of two possible output strings, the output string further away from zero is chosen.

Optionally, an [`IOStream`](@ref) may be passed as the first argument to redirect output.

# Examples

```jldoctest
julia> @printf("%f %F %f %F\n", Inf, Inf, NaN, NaN)
Inf Inf NaN NaN


julia> @printf "%.0f %.1f %f\n" 0.5 0.025 -0.0078125
1 0.0 -0.007813
```


In [5]:
function printtime(time)
    @printf("%02d:%02d:%02d", time.hour, time.minute, time.second)
end

printtime (generic function with 1 method)

In [6]:
mt = MyTime(2, 30, 40)

printtime(mt)

02:30:40

### Exercise 16-2

Write a boolean function called isafter that takes two MyTime objects, t1 and t2 , and returns true if t1 follows t2
chronologically and false otherwise. Challenge: don’t use an if statement.

* Relational operators work with tuples and other sequences.  Julia starts by comparing the first element from each sequence.  If they are equal, then it goes on to the next element, and so on, until it finds elements that differ.  Subsequent elements are not considered.

In [7]:
function isafter(t1, t2)
  (t1.hour, t1.minute, t1.second) > (t2.hour, t2.minute, t2.second)
end

isafter (generic function with 1 method)

In [8]:
mt1 = MyTime(10, 20, 30)
mt2 = MyTime(10, 21, 29)
isafter(mt1, mt2)

false

### Pure function and Modifiers

* A pure function does not modify any of the objects passed to it as arguments and it has no effects other than returning a value.

* A modifier changes the objects it gets as arguments.

### Exercise 16-3

In a development plan called prototype and patch, which is a way of tackling a complex problem by starting with a simple prototype and incrementally dealing with the complications.

The following function is a prototype intended to increment time of a MyTime instance:

```
function increment!(time,seconds)
    
    time.second += seconds
    
    if time.second >= 60
        time.second -= 60
        time.minute += 1
    end
    
    if time.minute >= 60
        time.minute -= 60
        time.hour += 1
    end
    
end
```

Is this function correct? What happens if seconds is much greater than 60?  In that case, it is not enough to carry once; we have to keep doing it until time.second is less than sixty. One solution is to replace the if statements with while statements. That would make the function correct, but not very efficient.

Write a correct version of increment! that doesn’t contain any loops.

In [10]:
?divrem

search: divrem



```
divrem(x, y)
```

The quotient and remainder from Euclidean division. Equivalent to `(div(x,y), rem(x,y))` or `(x÷y, x%y)`.

# Examples

```jldoctest
julia> divrem(3,7)
(0, 3)

julia> divrem(7,3)
(2, 1)
```


In [11]:
function increment!(time,seconds)
    
    (add_to_minute, new_second) = divrem(time.second + seconds, 60)
    (add_to_hour, new_minute) = divrem(time.minute + add_to_minute, 60)
    
    time.second = new_second
    time.minute = new_minute
    time.hour += add_to_hour
    
end 
    

increment! (generic function with 1 method)

In [33]:
mt3 = MyTime(10,20,50)
increment!(mt3, 220)
mt3

MyTime(10, 24, 30)

Anything that can be done with modifiers can also be done with pure functions. In fact, some programming languages only allow pure functions. There is some evidence that programs that use pure functions are faster to develop and less errorprone than programs that use modifiers. But modifiers are convenient at times, and functional programs tend to be less efficient.

In general, you are recommended to write pure functions whenever it is reasonable and resort to modifiers only if there is a
compelling advantage. This approach might be called a *functional programming style*.

### Exercise 16-4

Write a “pure” version of increment that creates and returns a new MyTime object rather than modifying the parameter.

In [13]:
function increment(time,seconds)
    
    (add_to_minute, new_second) = divrem(time.second + seconds, 60)
    (add_to_hour, new_minute) = divrem(time.minute + add_to_minute, 60)
    
    MyTime(time.hour + add_to_hour, new_minute, new_second)
    
end 

increment (generic function with 1 method)

In [14]:
mt4 = MyTime(10,20,50)
mt5 = increment(mt4, 220)

MyTime(10, 24, 30)

### Prototyping Versus Planning

“Prototype and patch” approach can be effective, especially if you don’t yet have a deep understanding of the problem. But incremental corrections can generate code that is unnecessarily complicated—since it deals with many special cases—and unreliable — since it is hard to know if you have found all the errors.

An alternative is designed development, in which high-level insight into the problem can make the programming much
easier. In this case, the insight is that a Time object is really a three-digit number in base 60.

This observation suggests another approach to the whole problem—we can convert MyTime objects to integers and take
advantage of the fact that the computer knows how to do integer arithmetic.

Ironically, sometimes making a problem harder (or more general) makes it easier (because there are fewer special cases and fewer opportunities for error).

### Exercise 16-5

Now, let's rewrite the function "increment!" using functions that covert a mytime to an integer and vice versa.

In [16]:
function timetoint(time)
    
    minutes = time.hour * 60 + time.minute
    seconds = minutes * 60 + time.second
    
end


function inttotime(seconds)
    
    (minutes, second) = divrem(seconds, 60)
    (hour, minute) = divrem(minutes, 60)
    MyTime(hour, minute, second)
    
end


inttotime (generic function with 1 method)

In [18]:
timetoint(inttotime(1240))  # consistency check

1240

In [19]:
function increment2!(time, seconds)
    
    new_time = inttotime( timetoint(time) + seconds )
    
    #(time.hour, time.minute, time.second) = (new_time.hour, new_time.minute, new_time.second)
    
    # I wanted to do it in a bit fancier way
    for f in propertynames(time)
        setproperty!( time, f, getproperty(new_time,f) )
    end
    
    #time = new_time # This does NOT work as a modifier as the new assignment simply creates a new MyTime instance
    
end

increment2! (generic function with 1 method)

In [21]:
fieldnames(MyTime)

(:hour, :minute, :second)

In [73]:
mt6 = MyTime(10,20,50)
increment2!(mt6, 220)
mt6

MyTime(10, 24, 30)

### Debugging

A MyTime object is well-formed if the values of minute and second are between 0 and 60 (including 0 but not 60) and if hour is positive. hour and minute should be integral values, but we might allow second to have a fraction part.

Requirements like these are called invariants because they should always be true. To put it a different way, if they are not true, something has gone wrong. Writing code to check invariants can help detect errors and find their causes. For example, you might have a function like isvalidtime that takes a MyTime object and returns false if it violates an invariant:

In [24]:
function isvalidtime(time)
    
    if time.hour < 0 || time.minute < 0 || time.second < 0
        return false
    end
    
    if time.minute >= 60 || time.second >= 60
        return false
    end
    
    true
    
end

isvalidtime (generic function with 1 method)

At the beginning of each function you could check the arguments to make sure they are valid:

In [25]:
function addtime(t1, t2)
    
    if !isvalidtime(t1) || !isvalidtime(t2)
        error("invalid MyTime object in add_time")
    end
    
    seconds = timetoint(t1) + timetoint(t2)
    inttotime(seconds)
    
end

addtime (generic function with 1 method)

Or you could use an @assert macro, which checks a given invariant and throws an exception if it fails:

In [26]:
function addtime(t1, t2)
    
    @assert(isvalidtime(t1) && isvalidtime(t2), "invalid MyTime object in add_time")
    seconds = timetoint(t1) + timetoint(t2)
    inttotime(seconds)
    
end

addtime (generic function with 1 method)